## Laden standard Packages

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.dates as dates
%matplotlib inline
import math
from dateutil.relativedelta import relativedelta

In [ ]:
np.set_printoptions(threshold=100)

## Datenaufbereitung


In [ ]:
# Laden der Daten
import os

data_path='/domino/datasets/local/TestData/iboxx_eur/'

In [ ]:
## Komponenten als monatsweises Dictionary von DataFrames 

comp_list=os.listdir(data_path+'Components/')

comp={}

for file in comp_list:
    comp[file[-10:-4]]=pd.read_csv(data_path+'Components/'+file,encoding='latin_1')

In [ ]:
## Indizes als ein GesamtDataframe auf täglicher Basis

indices_list=os.listdir(data_path+'Indices')

indices={}

for file in indices_list:
    indices[file[-12:-4]]=pd.read_csv(data_path+'Indices/'+file,encoding='latin_1')

i=0

for days in indices:
    if i==0:
        indices_df=pd.DataFrame(indices[days])
    else:
        indices_df=indices_df.append(indices[days],ignore_index=True)
    i=i+1

In [ ]:
## Underlyings als ein GesamtDataframe auf täglicher Basis

underlyings_list=os.listdir(data_path+'Underlyings')

underlyings={}

for file in underlyings_list:
    underlyings[file[-12:-4]]=pd.read_csv(data_path+'Underlyings/'+file,encoding='latin_1')

## Underlyings als ein GesamtDataframe auf täglicher Basis

underlyings_list=os.listdir(data_path+'Underlyings')

underlyings={}

for file in underlyings_list:
    underlyings[file[-12:-4]]=pd.read_csv(data_path+'Underlyings/'+file,encoding='latin_1')
    
underlyings_df=pd.DataFrame()

for days in underlyings:
    underlyings_df=underlyings_df.append(underlyings[days][['Date','ISIN','Index Price','Daily Return']],ignore_index=True)

### SLAIT - Specialized LAIT with upper bound

In [ ]:
def a_HETE(x,M=0.1):
    if abs(x)<=M:
        return 1
    else:
        return M/abs(x)

def b_HETE(x,M=0.1):
    if abs(x)<=M:
        return 0
    else:
        return M(abs(x)-M)

def b_HDR(x,M=0.1):
    if x<=M:
        return 0
    else:
        return M(abs(x)-M)
    
def a_HDR(x,M=0.1):
    if x<0:
        return M/(M-2*x)
    elif x<=M:
        return 1
    else:
        return M/abs(x)

def c(x):
    if x<0:
        return x
    else:
        return 0

def d (gamma,p,w):
    return 1/math.log(1+gamma/p)/(p+w)
    
def A(q,u=1,maxiter=1000):
    print('Start binary search')
    
    # indirect sort of q
    q_sortindex=q.argsort()
    
    i=1
    a=1
    e=len(q)
    K=(a+e)//2
    mue=-(q[q_sortindex[range(K-1)]].sum()+2)/(K)
    A=(mue+q)<0
    w=-(mue*np.ones(len(q))+q)/2
    #print(A.sum(),'==',K,'-- a=',a,' und e=',e)
        
    while (A.sum()!=K) & (i<=maxiter):
        i=i+1
        if A.sum()>K:
            a=K+1
            K=(a+e)//2
        else:
            e=K-1
            K=(a+e)//2
        mue=-(q[q_sortindex[range(K-1)]].sum()+2)/(K)
        A=(mue+q)<0
        #print(A.sum(),'==',K,'-- a=',a,' und e=',e)
        if A.sum()==K or e<a or i>maxiter:
            w=-(mue*np.ones(len(q))+q)/2
            break
        elif A.sum()==0:
             raise ValueError('Only zero weights')
        
    print('Found optimal K without upper bound. ',(w>u).sum(),' components above upper bound')
        
    if (u<1) & (u>0) & ((w>u).sum()>0):
        
        print('Start combined binary search')
        
        if 1/u>len(q):
            raise ValueError('upper bound too low')
            
        c=q+2*u
        
        for k in np.arange(K,len(q)+1):
            #print('K_opt == ',k)
            i=1
            a=1
            e=k
            K1=(a+e)//2
            K2=k-K1            
        
            mue=-(q[q_sortindex[K1:k-1]].sum()-2*K1*u+2)/K2
            B1=(mue+c)<=0
            B2=(0<mue+c) & (mue+c<2*u)
            
            #print(B1.sum(),'==',K1,' -- ',B2.sum(),'==',K2,'-- a=',a,' und e=',e)

            while (a<=e):
                
                i=i+1
                
                if B1.sum()>K1:
                    a=K1+1
                    K1=(a+e)//2
                    K2=k-K1 
                else:
                    e=K1-1
                    K1=(a+e)//2
                    K2=k-K1 
                
                mue=-(q[q_sortindex[K1:k-1]].sum()-2*K1*u+2)/K2
                B1=(mue+c)<=0
                B2=(0<mue+c) & (mue+c<2*u)
                
                #print(B1.sum(),'==',K1,' -- ',B2.sum(),'==',K2,'-- a=',a,' und e=',e)

                if (B1.sum()==K1) & (B2.sum()==K2):
                    w=np.minimum(-(mue*np.ones(len(q))+q)/2,u*np.ones(len(q)))
                    break
                #elif (B1.sum()==0) | (B2.sum()==0):
                    #raise ValueError('Only zero weights')
            
            if (B1.sum()==K1) & (B2.sum()==K2):
                break
        if ((B1.sum()!=K1) or (B2.sum()!=K2)):
            w[q_sortindex[range(math.ceil(1/u))]]=u
            w=w/w.sum()
    
    return np.maximum(w,0)

av_HETE=np.vectorize(a_HETE)
av_HDR=np.vectorize(a_HDR)
bv_HETE=np.vectorize(b_HETE)
bv_HDR=np.vectorize(b_HDR)
cv=np.vectorize(c)
dv=np.vectorize(d) 

In [ ]:
def SLAIT(X,r,lamb=1e-7,u=1,maxiter=1000,measure='ETE',eps=1e-9,w0=None,M=None,gamma=0.2,p=1e-6,thres=1e-3,ret=False):
    
    ################# error control ########################
    IDs=X.keys()
    X=np.array(X)
    r=np.array(r)
    T,n=X.shape # T is days, n number of constituents
    if n==1:
        raise ValueError('Data is univariate!')
    if pd.isna(X).sum()>0|pd.isna(r).sum()>0|pd.isna(lamb)|pd.isna(u):
        raise ValueError('Some arguments contain NaNs')
    if (measure in ['HETE','HDR'])&(pd.isnull(M)|(M<=0)):
        raise ValueError('The huber parameter should be positive')
    if (u>1)|(u<=0):
        raise ValueError('upper bound not in range (0,1]')
    if measure not in ['ETE','HETE','DR','HDR']:
        raise ValueError(measure,' is no valid measure!')
    ########################################################
    
    if w0 is None:
        w=np.ones(n) 
        w=w/w.sum() # equal weights for every component
    else:
        if np.isnan(w0).any():
            raise ValueError('NaNs in start vector')
        else:
            w=w0
    
    k=0
    conv=eps
    
    if measure in ['ETE','DR']: 
        L1=(X.T@X)/T
        lamb_max=np.linalg.eigvalsh(L1).max()
    
    elif measure in ['HETE','HDR']:
        av=av_HETE(r-X@w)
        aDiag=np.eye(T)
        np.fill_diagonal(aDiag,av)
        L2=(X.T@aDiag@X)/T
        lamb_max=np.linalg.eigvalsh(L2).max()
    
    while (conv>=eps) & (k<=maxiter):
        if measure=='ETE':
            q=(2*(L1-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@r)/lamb_max
        elif measure=='DR':
            y=-np.maximum(X@w-r,0)
            q=(2*(L1-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@(y-r))/lamb_max
        elif measure=='HETE':
            av=av_HETE(r-X@w)
            aDiag=np.eye(T)
            np.fill_diagonal(aDiag,av)
            q=(2*(L2-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@aDiag@r)/lamb_max
        else:
            av=av_HDR(r-X@w)
            aDiag=np.eye(T)
            np.fill_diagonal(aDiag,av)
            L3=X.T@aDiag@X/T
            lamb_max=np.linalg.eigvalsh(L3).max()
            q=(2*(L3-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@aDiag@(cv(r-X@w-r)))/lamb_max
        w0=w
        w=A(q,u,maxiter)
        #conv=norm(w0-w)
        k=k+1
        
        #Needs python 3.10 
        #match measure:
            #case 'ETE':
            #    print('Iteration: ',k,'--> Zielfunktion: ',ETE(X,r,w),' and w: ',w)
            #case 'DR':
            #    print('Iteration: ',k,'--> Zielfunktion: ',DR(X,r,w),' and w: ',w)
            #case 'HETE':
            #    print('Iteration: ',k,'--> Zielfunktion: ',HETE(X,r,w,M),' and w: ',w)
            #case 'HDR':
            #    print('Iteration: ',k,'--> Zielfunktion: ',HDR(X,r,w,M),' and w: ',w)
        
        if measure=='ETE':
            print('Iteration: ',k,'Komponenten: ',(w>0).sum(),'--> Zielfunktion: ',ETE(X,r,w)+dv(gamma,p,w).T@w, \
                  'Maß: ',ETE(X,r,w),' and w: ',w)
            conv=abs(ETE(X,r,w0)-ETE(X,r,w))+abs(dv(gamma,p,w0).T@w0-dv(gamma,p,w).T@w)
        elif measure=='DR':
            print('Iteration: ',k,'Komponenten: ',(w>0).sum(),'--> Zielfunktion: ',DR(X,r,w)+dv(gamma,p,w).T@w, \
                  'Maß: ',DR(X,r,w),' and w: ',w)
            conv=abs(DR(X,r,w0)-DR(X,r,w))+abs(dv(gamma,p,w0).T@w0-dv(gamma,p,w).T@w)
        elif measure=='HETE':
            print('Iteration: ',k,'Komponenten: ',(w>0).sum(),'--> Zielfunktion: ',HETE(X,r,w,M)+dv(gamma,p,w).T@w, \
                  'Maß: ',HETE(X,r,w,M),' and w: ',w)
            conv=abs(HETE(X,r,w0)-HETE(X,r,w))+abs(dv(gamma,p,w0).T@w0-dv(gamma,p,w).T@w)
        else:
            print('Iteration: ',k,'Komponenten: ',(w>0).sum(),'--> Zielfunktion: ',HDR(X,r,w,M)+dv(gamma,p,w).T@w,\
                  'Maß: ',HDR(X,r,w,M),' and w: ',w)
            conv=abs(HDR(X,r,w0)-HDR(X,r,w))+abs(dv(gamma,p,w0).T@w0-dv(gamma,p,w).T@w)
            
    if w.max()>thres:
        w=np.where(w<thres,0,w)
    w=np.nan_to_num(w)
    w=w/w.sum()
    
    W=pd.DataFrame({'ISIN':IDs,'Weight':w})
                
    return W

def ETE(X,r,w):
    T,n=X.shape
    return norm(X@w-r)**2/T

def DR(X,r,w):
    T,n=X.shape
    return norm(np.maximum(r-X@w,0))**2/T

def phi(x,M):
    if x<=abs(M):
        return x**2
    else:
        return M*(2*abs(x)-M)
    
phiv=np.vectorize(phi)

def HETE(X,r,w,M):
    T,n=X.shape
    return np.ones(T).T@phiv(X@w-r,M)/T

def HDR(X,r,w,M):
    T,n=X.shape
    return np.ones(T).T@phiv(np.maximum(r-X@w,0),M)/T

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
def plot_replicated(w,Index,Comp,Ret=True,Start=100):
    plt.figure(i,figsize=(15,5))

    
    return_reb=Comp@w
    return_org=Index
    
    r=range(len(return_reb))
    
    if Ret==False:
        price_reb=return_reb
        price_org=return_org
        return_reb=np.zeros(len(price_reb))
        return_org=np.zeros(len(price_reb))
        for j in r:
            if j==0:
                return_reb[j]=Start
                return_org[j]=Start
            else:
                return_reb[j]=return_reb[j-1]*(1+np.log(price_reb[j]/price_reb[j-1]))
                return_org[j]=return_org[j-1]*(1+np.log(price_org[j]/price_org[j-1]))
    r=np.array(Index.index)
    plt.plot(r,return_reb, label='rebuild')
    plt.plot(r,return_org, label='original')
            
    plt.legend()

In [ ]:
class portfolio (object):
    def __init__(self,weight=None,underlying=None,index=None,components=None):
            self.weight=weight
            self.underlying=underlying
            self.index=index
            self.components=components
            
    def plt_replicated(self,Start=100):
        plt.figure(i,figsize=(15,5))
        
        data=pd.merge_asof(self.underlying,self.weight,on='Date',by='ISIN',direction='backward',allow_exact_matches=False) \
        .dropna()
        data['WeightReturn']=data['Weight']*data['Daily Return']
        data=data.groupby('Date').sum()
        data=pd.merge(data,self.index,on='Date')
        
    
        return_reb=np.array(data['WeightReturn'])
        return_org=np.array(data['Daily Return_y'])
    
        r=range(len(return_reb))
    
        for j in range(len(return_reb)):
            if j==0:
                return_reb[j]=Start
                return_org[j]=Start
            else:
                return_reb[j]=return_reb[j-1]*(1+return_reb[j])
                return_org[j]=return_org[j-1]*(1+return_org[j])
        r=data.index
        plt.plot(r,return_reb, label='rebuild')
        plt.plot(r,return_org, label='original')

        plt.legend()
        plt.show()
        
    def number_components(self):
        return self.weight[self.weight['Weight']>0][['Date','ISIN']].groupby('Date').count()
    
    def compare(self,on,date=None):
        if pd.isna(date):
            date=self.weight['Date'].max()
        data=pd.merge(self.components[date.strftime('%Y%m')],self.weight[self.weight['Date']==date],on='ISIN',how='left').fillna(0)
        data['Index Weight']=data['Index Weight']/data['Index Weight'].sum()
        if self.components[date.strftime('%Y%m')][on].dtype=='float64':
            data[on]=round(data[on],0)
            print('Rebuild ',on,': ',np.average(data[on], weights=data['Weight']))
            print('Index ',on,': ',np.average(data[on], weights=data['Index Weight']))
        data.groupby(on).sum()[['Weight','Index Weight']].plot(kind='bar',figsize=(25,5))
    
    def compare_list(self):
        return list(self.components[list(self.components.keys())[0]].keys())
    
    def ETE(self,ret=True,Start=1):
        data=pd.merge_asof(self.underlying,self.weight,on='Date',by='ISIN',direction='backward',allow_exact_matches=False) \
        .dropna()
        data['WeightReturn']=data['Weight']*data['Daily Return']
        data=data.groupby('Date').sum()
        data=pd.merge(data,self.index,on='Date')
        return_reb=np.array(data['WeightReturn'])
        return_org=np.array(data['Daily Return_y'])
        
        # Vergleich auf Preisbasis
        if ret==False:
            for j in range(len(return_reb)):
                if j==0:
                    return_reb[j]=Start
                    return_org[j]=Start
                else:
                    return_reb[j]=return_reb[j-1]*(1+return_reb[j])
                    return_org[j]=return_org[j-1]*(1+return_org[j])
                
        return norm(return_reb-return_org)**2/len(return_reb)
    
    def DR(self,ret=True,Start=1):
        data=pd.merge_asof(self.underlying,self.weight,on='Date',by='ISIN',direction='backward',allow_exact_matches=False) \
        .dropna()
        data['WeightReturn']=data['Weight']*data['Daily Return']
        data=data.groupby('Date').sum()
        data=pd.merge(data,self.index,on='Date')
        return_reb=np.array(data['WeightReturn'])
        return_org=np.array(data['Daily Return_y'])
        
        # Vergleich auf Preisbasis
        if ret==False:
            for j in range(len(return_reb)):
                if j==0:
                    return_reb[j]=Start
                    return_org[j]=Start
                else:
                    return_reb[j]=return_reb[j-1]*(1+return_reb[j])
                    return_org[j]=return_org[j-1]*(1+return_org[j])
                
        return norm(np.maximum(return_org-return_reb,0))**2/len(return_org)
    
    def trading_cost(self,base=100,c=5e-4):
        data=self.weight.pivot(columns='Date',values='Weight',index='ISIN').fillna(0)
        
        for (i,date) in enumerate(data.columns):
            if i==0:
                cost=0
            cost=cost+c*norm(data.iloc[:,i]-data.iloc[:,i-1])            
                
        return cost*base
    
    def bid_ask_trading(self, volume=1, c_fix=5e-4,prc_factor=100):
        data=self.weight.pivot(columns='Date',values='Weight',index='ISIN').fillna(0)
        
        for (i,date) in enumerate(data.columns):
            comp_price=pd.merge(self.components[date.strftime('%Y%m')][['ISIN','Bid Price', 'Ask Price', 'Index Price']] \
                                ,data[date],on='ISIN')
            comp_price['Ask Nom']=comp_price.iloc[:,4]*volume/comp_price['Ask Price']*prc_factor
            comp_price['Bid Nom']=comp_price.iloc[:,4]*volume/comp_price['Bid Price']*prc_factor
            comp_price['Index Nom']=comp_price.iloc[:,4]*volume/comp_price['Index Price']*prc_factor
            
            if i==0:
                Nom=pd.DataFrame(comp_price['Ask Nom']).rename_axis(date)
                Prc=pd.DataFrame(comp_price['Ask Price']).rename_axis(date)
                value=volume-(Nom*Prc/prc_factor).sum()-(Nom*c_fix).sum()
                value_series={date:value}
            else:
                Nom[date]=np.where(Nom.iloc[:,i-1]-comp_price['Index Nom']<0,comp_price['Ask Nom'],comp_price['Bid Nom'])
                Prc[date]=np.where(Nom.iloc[:,i-1]-comp_price['Index Nom']<0,comp_price['Ask Price'],comp_price['Bid Price'])               
                value=value+((Nom.iloc[:,i-1]-Nom.iloc[:,i])*Prc[date]/prc_factor).sum() \
                            -(abs(Nom.iloc[:,i-1]-Nom.iloc[:,i])*c_fix).sum()
                value_series={date:value}
        
            return value,value_series,Nom,Prc
            
    def normalize_weight(self,nominal=None,min_piece=None,weight_step=None,inplace=False):
        if not pd.isna(nominal) or not pd.isna(min_piece):
            weight_step=min_piece/nominal
        elif pd.isna(weight_step):
            raise ValueError('No argument given')
            
        data=self.weight.pivot(columns='Date',values='Weight',index='ISIN').fillna(0)
        
        for (i,date) in enumerate(data.columns):
            if i==0:
                data[date]=(round(data[date]/weight_step,0))*weight_step
            else:
                weight_new=(round(data[date]/weight_step))*weight_step
                data.iloc[:,i]=np.where(abs(data.iloc[:,i]-data.iloc[:,i-1])>weight_step,data.iloc[:,i],data.iloc[:,i-1])
                #data.iloc[:,i-1]+((data.iloc[:,i]-data.iloc[:,i-1])//weight_step)*weight_step
        
        data=data.reset_index().melt(value_vars=data.columns,id_vars='ISIN',value_name='Weight')\
            .dropna().reset_index(drop=True)
        if inplace:
            self.weight=data[['ISIN','Weight','Date']]
        else:
            return data[['ISIN','Weight','Date']]
        

In [ ]:
def MultiPeriod_SLAIT(X,r,Comp,IndexISIN='DE0009682716',Startdate=None,Enddate=None,m_rebal_freq=1,MonthRew=1, \
                      lamb=1e-7,u=1,maxiter=1000,measure='ETE' \
                      ,eps=1e-9,w0=None,M=0.1,gamma=0.2,p=1e-6,thres=1e-3,ret=False):

    # Datenvorbereitung
    r=r.sort_values('Date',ignore_index=True)
    X=X.sort_values(['Date','ISIN'],ignore_index=True)

    r=r.set_index(pd.to_datetime(r.Date)).drop('Date',axis=1)
    X=X.set_index(pd.to_datetime(X.Date)).drop('Date',axis=1)
    
    # Standardwert für Zeitraum -> Maximum möglich
    if Startdate==None:
        Startdate=X.index.min()
    if Enddate==None:
        Enddate=X.index.max()
    
    #Filter Indexdaten auf gewünschten Index
    r=r[r['ISIN_TRi']==IndexISIN]
        
    #Initialisierung leeres Portfolio
    port=pd.DataFrame()
    
    # Trennung in Teilzeiträume    
    dates=pd.date_range(Startdate,Enddate,freq='M')
    
    if len(dates)%m_rebal_freq!=0:
        raise ValueError('Number of dates available must be multiple of Month rebalance')
    
    # Iteration über Zeiträume
    for i in range(MonthRew-1,len(dates)-MonthRew,MonthRew):
        
        print('Starte Optimierung für ', dates[i])
        
        #Auswahl der Komponenten für kommenden Zeitraum 
        Components=Comp[dates[i+1].strftime('%Y%m')]
        print('Components: ',Components.shape)

        # Filter auf alle Komponentenzeitreihen des Vorzeitraumes und im neuen noch aktiv sind
        Underlying=X[(X.index>=dates[i-MonthRew+1].replace(day=1)) & (X.index<=dates[i])]
        Underlying=Underlying[Underlying['ISIN'].isin(list(Components['ISIN']))]
        
        #Bestimmung der Startgewichtung: entweder Startvektor oder aber die Gewichtung des Vorzeitraumes
        if port.empty:
            w=w0
        else:
            w=np.array(pd.merge(Underlying,port[port['Date']==port['Date'].max()],on='ISIN',how='left')\
                       .fillna(0).groupby('ISIN').mean()['Weight'])
        
        # Extraktion Return-Zeitreihen für Index und Underlyings
        Underlying=Underlying.pivot(columns='ISIN',values='Daily Return').fillna(0)
        print('Underlying: ',Underlying.shape)
        Index=r[(r.index>=dates[i-MonthRew+1].replace(day=1)) & (r.index<=dates[i])]
        Index=Index['Daily Return']
        print('Index: ',Index.shape)
        
        # Wenn Steuervariable ret = Falsch, dann Umwandlung DailyReturns in fortlaufende Returns
        Index_temp=pd.DataFrame(Index)
        Underlying_temp=pd.DataFrame(Underlying)
        if ret==False:
            for j in range(len(Index)):
                if j>0:
                    Index[j]=(1+Index[j-1])*(1+Index[j])-1
                    Underlying.iloc[j,:]=(1+Underlying.iloc[j-1,:])*(1+Underlying.iloc[j,:])-1
        
        result=SLAIT(X=Underlying,r=Index,u=u,measure=measure,eps=eps,M=M,maxiter=maxiter,lamb=lamb,w0=w,gamma=gamma,p=p,thres=thres)
        result['Date']=dates[i]
        result=result[result['Weight']>0]
                
        port=port.append(result,ignore_index=True)

    return portfolio(port,X,r,Comp)#,Underlying,Index,Underlying_temp,Index_temp

#### Einschränkung auf Komponenten

In [ ]:
for key in comp.keys():
    print(comp[key].shape , key)

In [ ]:
comp_corp=dict(comp)
for key in  comp.keys():
    act_comp=comp_corp[key]
    comp_corp[key]=act_comp[(act_comp['Level 2']=='Non-Financials') \
                      & (act_comp['Seniority Level 1']=='SEN') \
                      & (act_comp['Markit iBoxx Rating'].isin(['AAA','AA','A','BBB']))
                      & (act_comp['Bid_Ask_Spread']<0.5)]

In [ ]:
for key in comp_corp.keys():
    print(comp_corp[key].shape , key)

In [ ]:
W1_Corp=MultiPeriod_SLAIT(underlyings_df,indices_df,comp_corp,Startdate='2020-01-01',Enddate='2022-03-31',maxiter=10000,eps=1e-7,m_rebal_freq=3,MonthRew=3,u=0.02,IndexISIN='DE000A0G84N4',lamb=1e-8)

In [ ]:
W1_Corp.weight.to_csv('results/Result_CorpW1.csv')

In [ ]:
W2_Corp=MultiPeriod_SLAIT(underlyings_df,indices_df,comp_corp,Startdate='2020-01-01',Enddate='2022-05-31',maxiter=10000,eps=1e-7,m_rebal_freq=1,MonthRew=1,u=0.02,IndexISIN='DE000A0G84N4',lamb=1e-7)

In [ ]:
W2_Corp.weight.to_csv('results/Result_CorpW2.csv')

In [ ]:
W3_Corp=MultiPeriod_SLAIT(underlyings_df,indices_df,comp_corp,Startdate='2020-01-01',Enddate='2022-05-31',maxiter=10000,eps=1e-7,m_rebal_freq=1,MonthRew=1,u=0.02,IndexISIN='DE000A0G84N4',lamb=1e-8)
W3_Corp.weight.to_csv('results/Result_CorpW3.csv')

In [ ]:
W4_Corp=MultiPeriod_SLAIT(underlyings_df,indices_df,comp_corp,Startdate='2020-01-01',Enddate='2022-05-31',maxiter=10000,eps=1e-7,m_rebal_freq=1,MonthRew=3,u=0.02,IndexISIN='DE000A0G84N4',lamb=1e-8)
W4_Corp.weight.to_csv('results/Result_CorpW4.csv')

In [ ]:
W5_Corp=MultiPeriod_SLAIT(underlyings_df,indices_df,comp_corp,Startdate='2020-01-01',Enddate='2022-05-31',maxiter=10000,eps=1e-7,m_rebal_freq=1,MonthRew=3,u=0.01,IndexISIN='DE000A0G84N4',lamb=1e-7)
W5_Corp.weight.to_csv('results/Result_CorpW5.csv')